In [1]:
import pandas as pd
import glob

In [2]:
path = r'Desktop/sumoResults'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
sumoR = pd.concat(dfs, ignore_index=True)

In [3]:
for num in range(1983,2019):
    sumoD = pd.read_csv("Desktop/sumoDemo/" +str(num)+".csv")
    sumoD = sumoD.append(sumoD)

In [7]:
sumoR.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1990.01,1,1309,J13w,Tamakairiki,1-0 (8-7),True,yoritaoshi,1326,J13e,Nishinoryu,0-1 (10-5),False
1,1990.01,1,1326,J13e,Nishinoryu,0-1 (10-5),False,yoritaoshi,1309,J13w,Tamakairiki,1-0 (8-7),True
2,1990.01,1,1362,J12e,Onohana,1-0 (7-8),True,yorikiri,1369,J12w,Toryu,0-1 (4-11),False
3,1990.01,1,1369,J12w,Toryu,0-1 (4-11),False,yorikiri,1362,J12e,Onohana,1-0 (7-8),True
4,1990.01,1,1313,J11e,Komafudo,1-0 (7-8),True,yoritaoshi,1361,J11w,Daitetsu,0-1 (9-6),False


In [8]:
new = sumoR[['rikishi1_rank']].copy()
# <p><b>1</b>: A 1 or 2 letter code for the rikishi's division or makuuchi rank:\n</p>,
# <p><b>2</b>: Number within division or rank, or H for sanyaku and  maegashira haridashi<br/>\n<b>3</b>: <b>E</b> or <b>W</b> for East or West\n</p>]
new['rikishi1_rank'] = new['rikishi1_rank'].str.replace('\d+', '')
new['rikishi1_rank'] = new['rikishi1_rank'].str.replace('e', '')
new['rikishi1_rank'] = new['rikishi1_rank'].str.replace('w', '')
new=new.drop_duplicates()
new=new.rename(index=str, columns={"rikishi1_rank": "Rank_Code"})
new

,Rank_Code
0,J
26,M
50,S
53,OHD
54,O
58,SHD
59,YHD
60,K
61,Y
576,Ms


In [9]:
import requests
from bs4 import BeautifulSoup as bs

In [10]:
url="https://en.m.wikipedia.org/wiki/Template:Sumo_rank"
response = requests.get(url)
soup=bs(response.text,'html.parser')
thangs=soup.select('p')
thangs

[<p>This constructs a string for a sumotori's rank. It takes three unnamed parameters. All are optional, but if none are given the results are very uninteresting. It's primarily intended for use by other templates, but it may have a broader application.\n</p>,
 <p><b>1</b>: A 1 or 2 letter code for the rikishi's division or makuuchi rank:\n</p>,
 <p><b>2</b>: Number within division or rank, or H for sanyaku and  maegashira haridashi<br/>\n<b>3</b>: <b>E</b> or <b>W</b> for East or West\n</p>]

In [11]:
url="https://en.m.wikipedia.org/wiki/Template:Sumo_rank"
response = requests.get(url)
soup=bs(response.text,'html.parser')
things=soup.select('li')
things

[<li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-edit" id="ca-edit" title="Edit the lead section of this page"><a class="edit-page" href="/w/index.php?title=Template:Sumo_rank&amp;action=edit&amp;section=0">Edit</a></li>,
 <li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-watch watch-this-article" id="ca-watch"><a accesskey="w" href="/w/index.php?title=Special:UserLogin&amp;returnto=Template%3ASumo+rank" title="Add this page to your watchlist [w]"></a></li>,
 <li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-language-switcher language-selector" title="Read in another language"><a href="/wiki/Special:MobileLanguages/Template:Sumo_rank">Read in another language</a></li>,
 <li><b>Y</b>: Yokozuna</li>,
 <li><b>O</b>: \u014czeki</li>,
 <li><b>S</b>: Sekiwake</li>,
 <li><b>K</b>: Komosubi</li>,
 <li><b>M</b>: Maegashira</li>,
 <li><b>J</b>: J\u016bry\u014d</li>,
 <li><b>MA</b>: Makushita</li>,
 <li><b>MT</b>: Makushita tsukedashi</li>,
 <li><b>SA</b>: Sandanme

In [12]:
len(things)

21

In [13]:
rankdf=pd.DataFrame()
x=0
for row in things[1:]:
        rankdf.loc[x, 'Yokozuna'] = row.text
        x += 1

In [14]:
rankdf=rankdf.drop([0,1,17,18,19])

In [15]:
rankdf['rank'] = rankdf['Yokozuna'].apply(lambda s:s.split(':')[1])

In [16]:
rankdf['code'] = rankdf['Yokozuna'].apply(lambda s:s.split(':')[0])

In [17]:
rankdf.drop(columns=['Yokozuna'])

,rank,code
2,Yokozuna,Y
3,Ōzeki,O
4,Sekiwake,S
5,Komosubi,K
6,Maegashira,M
7,Jūryō,J
8,Makushita,MA
9,Makushita tsukedashi,MT
10,Sandanme,SA
11,Sandanme tsukedashi,ST


In [18]:
finaldf = pd.merge(rankdf, new,
                           left_on='code', right_on='Rank_Code',
                           how='outer'
                           ) 

In [19]:
finaldf

,Yokozuna,rank,code,Rank_Code
0,Y: Yokozuna,Yokozuna,Y,Y
1,O: Ōzeki,Ōzeki,O,O
2,S: Sekiwake,Sekiwake,S,S
3,K: Komosubi,Komosubi,K,K
4,M: Maegashira,Maegashira,M,M
5,J: Jūryō,Jūryō,J,J
6,MA: Makushita,Makushita,MA,NaN
7,MT: Makushita tsukedashi,Makushita tsukedashi,MT,NaN
8,SA: Sandanme,Sandanme,SA,NaN
9,ST: Sandanme tsukedashi,Sandanme tsukedashi,ST,NaN


In [20]:
finaldf=finaldf.dropna()

In [21]:
finaldf=finaldf[['rank', 'Rank_Code']].copy()
finaldf=finaldf.rename(index=str, columns={"rank": "Rank"})

finaldf

,Rank,Rank_Code
0,Yokozuna,Y
1,Ōzeki,O
2,Sekiwake,S
3,Komosubi,K
4,Maegashira,M
5,Jūryō,J


In [25]:
# SQL dependencies
# sqlite database file
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

engine = create_engine("sqlite:///n8ETL.sqlite")

In [26]:
finaldf.to_sql('Rank', con=engine)
sumoD.to_sql('Results', con=engine)
sumoR.to_sql('Demographics', con=engine)

In [36]:
import sqlite3

sqlite_file = 'n8ETL.sqlite'    # name of the sqlite database file
table_name = 'Rank'   # name of the table to be created
id_column = 'Rank_Code' # name of the PRIMARY KEY column
new_column = 'unique_names'  # name of the new column
column_type = 'TEXT' # E.g., INTEGER, TEXT, NULL, REAL, BLOB
index_name = 'my_unique_index'  # name for the new unique index

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Adding a new column and update some record
c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
        .format(tn=table_name, cn=new_column, ct=column_type))
c.execute("UPDATE {tn} SET {cn}='sebastian_r' WHERE {idf}=123456".\
        format(tn=table_name, idf=id_column, cn=new_column))

# Creating an unique index
c.execute('CREATE INDEX {ix} on {tn}({cn})'\
        .format(ix=index_name, tn=table_name, cn=new_column))

# Dropping the unique index
# E.g., to avoid future conflicts with update/insert functions
c.execute('DROP INDEX {ix}'.format(ix=index_name))

# Committing changes and closing the connection to the database file
conn.commit()
conn.close()

OperationalError: duplicate column name: unique_names

In [38]:
import sqlite3


def connect(sqlite_file):
    """ Make connection to an SQLite database file """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c


def close(conn):
    """ Commit changes and close connection to the database """
    # conn.commit()
    conn.close()


def total_rows(cursor, table_name, print_out=False):
    """ Returns the total number of rows in the database """
    cursor.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cursor.fetchall()
    if print_out:
        print('\nTotal rows: {}'.format(count[0][0]))
    return count[0][0]


def table_col_info(cursor, table_name, print_out=False):
    """ Returns a list of tuples with column informations:
    (id, name, type, notnull, default_value, primary_key)
    """
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()

    if print_out:
        print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
        for col in info:
            print(col)
    return info


def values_in_col(cursor, table_name, print_out=True):
    """ Returns a dictionary with columns as keys
    and the number of not-null entries as associated values.
    """
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()
    col_dict = dict()
    for col in info:
        col_dict[col[1]] = 0
    for col in col_dict:
        c.execute('SELECT ({0}) FROM {1} '
                  'WHERE {0} IS NOT NULL'.format(col, table_name))
        # In my case this approach resulted in a
        # better performance than using COUNT
        number_rows = len(c.fetchall())
        col_dict[col] = number_rows
    if print_out:
        print("\nNumber of entries per column:")
        for i in col_dict.items():
            print('{}: {}'.format(i[0], i[1]))
    return col_dict


if __name__ == '__main__':

    sqlite_file = 'n8ETL.sqlite'
    table_name = 'Rank'

    conn, c = connect(sqlite_file)
    total_rows(c, table_name, print_out=True)
    table_col_info(c, table_name, print_out=True)
    # next line might be slow on large databases
    values_in_col(c, table_name, print_out=True)

    close(conn)


Total rows: 6

Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, u'index', u'TEXT', 0, None, 0)
(1, u'Rank', u'TEXT', 0, None, 0)
(2, u'Rank_Code', u'TEXT', 0, None, 0)
(3, u'unique_names', u'TEXT', 0, None, 0)


OperationalError: near "index": syntax error